In [2]:
import os
from pathlib import Path
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
import zipfile
import gdown
from cnnClassifier import logger

/workspaces/Kidney-Disease-Classification/research
Target path: config/config.yaml
Exists: False
Target path: params.yaml
Exists: False


In [3]:
from cnnClassifier.utils.common import *

In [4]:
print(Path().cwd())

/workspaces/Kidney-Disease-Classification/research


In [5]:
current_dir = Path().cwd()

In [6]:
print(CONFIG_FILE_PATH.exists())

False


In [7]:
current_dir

PosixPath('/workspaces/Kidney-Disease-Classification/research')

In [8]:
current_dir.parent

PosixPath('/workspaces/Kidney-Disease-Classification')

In [9]:
print(Path().cwd())

/workspaces/Kidney-Disease-Classification/research


In [10]:
print(current_dir.parent)

/workspaces/Kidney-Disease-Classification


In [11]:
config_path = current_dir.parent / CONFIG_FILE_PATH

In [12]:
params_path = current_dir.parent / PARAMS_FILE_PATH

In [13]:
config_path

PosixPath('/workspaces/Kidney-Disease-Classification/config/config.yaml')

In [14]:
config_path.exists(), params_path.exists()

(True, True)

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url:str
    local_data_file: Path
    unzip_dir: Path

In [16]:
import yaml
from box import ConfigBox
from box.exceptions import BoxValueError

In [17]:
with open(config_path) as config_yaml_file:
    print(config_yaml_file)

<_io.TextIOWrapper name='/workspaces/Kidney-Disease-Classification/config/config.yaml' mode='r' encoding='UTF-8'>


In [18]:
with open(config_path) as config_yaml_file:
    content = yaml.safe_load(config_yaml_file)
    print(content)

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}


In [19]:
try:
    with open(config_path) as yaml_file:
        content = yaml.safe_load(yaml_file)
        logger.info(f"yaml file: {config_path} loaded successfully")
        print(ConfigBox(content))
except BoxValueError:
    raise ValueError("yaml file is empty")
except Exception as e:
    raise e

[2026-01-09 10:58:12,676: INFO: 927828566: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}


In [20]:
config = read_yaml(config_path)

[2026-01-09 10:58:12,700: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]


In [21]:
config.artifacts_root

'artifacts'

In [22]:
create_directories([config.artifacts_root])

[2026-01-09 10:58:12,722: INFO: common: created directory at: artifacts]


In [23]:
create_directories([config.data_ingestion.root_dir])

[2026-01-09 10:58:12,732: INFO: common: created directory at: artifacts/data_ingestion]


In [24]:
class ConfigurationManager:
    def __init__(self, config_filepath = config_path, params_filepath = params_path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [25]:
config_manager = ConfigurationManager()

[2026-01-09 10:58:12,772: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/config/config.yaml loaded successfully]
[2026-01-09 10:58:12,775: INFO: common: yaml file: /workspaces/Kidney-Disease-Classification/params.yaml loaded successfully]
[2026-01-09 10:58:12,777: INFO: common: created directory at: artifacts]


In [26]:
data_ingestion_config = config_manager.get_data_ingestion_config()

[2026-01-09 10:58:12,796: INFO: common: created directory at: artifacts/data_ingestion]


In [27]:
data_ingestion_config

DataIngestionConfig(root_dir='artifacts/data_ingestion', source_url='https://drive.google.com/file/d/1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA/view?usp=sharing', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')

In [28]:
file_id = data_ingestion_config.source_url.split('/')[-2]
file_id

'1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA'

In [29]:
# https://drive.google.com/file/d/1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA/view?usp=sharing

In [30]:
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, data_ingestion_config.local_data_file)

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA&confirm=t&uuid=7a10fdb7-1959-481b-9996-e82830cc6944
To: /workspaces/Kidney-Disease-Classification/research/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:03<00:00, 18.8MB/s]


'artifacts/data_ingestion/data.zip'

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try: 
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.unzip_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [32]:
data_ingestion = DataIngestion(data_ingestion_config)

In [33]:
data_ingestion.download_file()

[2026-01-09 10:58:19,741: INFO: 4226074977: Downloading data from https://drive.google.com/file/d/1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA&confirm=t&uuid=c3097119-9ba1-43bb-a0cb-87a22e5063ea
To: /workspaces/Kidney-Disease-Classification/research/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:02<00:00, 24.2MB/s]

[2026-01-09 10:58:24,518: INFO: 4226074977: Downloaded data from https://drive.google.com/file/d/1gK9DTZ1ReF2a-LX1N0fEveYbQUTXDaHA/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [37]:
data_ingestion.extract_zip_file()

In [34]:
import os

path = "config/config.yaml"
print(f"Checking path: {os.path.abspath(path)}")
print(f"Does the folder exist? {os.path.exists('config')}")
print(f"Does the file exist? {os.path.exists(path)}")

Checking path: /workspaces/Kidney-Disease-Classification/research/config/config.yaml
Does the folder exist? False
Does the file exist? False


In [35]:
os.getcwd()

'/workspaces/Kidney-Disease-Classification/research'

In [36]:
# try:
#     config = ConfigurationManager()
#     data_ingestion_config = config.get_data_ingestion_config()
#     data_ingestion = DataIngestion(config=data_ingestion_config)
#     data_ingestion.download_file()
#     data_ingestion.extract_zip_file()
# except Exception as e:
#     raise e